In [1]:
#import library
from bs4 import BeautifulSoup
import requests

import re

import numpy as np
import pandas as pd

import os
import base64
from requests import post, get

import json
import time

from datetime import datetime

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

import math

import smtplib
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
input_file_system_name = 'datasets'
storage_account_name = 'spotifyprojectadls'
adls_path = f"abfss://{input_file_system_name}@{storage_account_name}.dfs.core.windows.net/"

In [3]:
# Start the timer
starter = time.time()

# **1. Access to Spotify API**

In [4]:
client_id = 'abcdef1234'
client_secret = 'abcdef1234'

def get_token():
    url = 'https://accounts.spotify.com/api/token'
    
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    headers = {
        'content-type': 'application/x-www-form-urlencoded',
        'Authorization': 'Basic ' + auth_base64
    }

    form = {
        'grant_type': 'client_credentials'
    }

    result = post(url, headers=headers, data=form)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token

def get_auth_header(token):
    return {"Authorization": f"Bearer {token}"}

# **2. Today Top 200 Track Update**

In [5]:
# Scrape for top 200 track daily and add into the previous one
top_200_track_daily = pd.read_parquet(adls_path + 'top_200_track_daily.parquet')

url = 'https://kworb.net/spotify/country/global_daily.html'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')
col_data = soup.find_all('tr')

for row in col_data[1:]:
    row_data = row.find_all('td')

    track_url = row_data[2].find_all('a')[1].get('href')
    artist_url = row_data[2].find_all('a')[0].get('href')

    daily_stream = row_data[-5].text.strip()
    daily_stream = daily_stream.replace(",","")
    daily_stream = int(daily_stream)

    total_stream = row_data[-1].text.strip()
    total_stream = total_stream.replace(",","")
    total_stream = int(total_stream)

    track_pattern = r'../track/([^_]+)\.html'
    track_match = re.search(track_pattern, track_url)
    if track_match:
        track_id = track_match.group(1)
    else:
        track_id = np.NaN

    artist_pattern = r'../artist/([^_]+)\.html'
    artist_match = re.search(artist_pattern, artist_url)
    if artist_match:
        artist_id = artist_match.group(1)
    else:
        artist_id = np.NaN

    now = datetime.now()
    extract_date = now.strftime("%Y-%m-%d %H:%M:%S")

    individual_row_data = [track_id, artist_id, daily_stream, total_stream, extract_date]

    length = len(top_200_track_daily)
    top_200_track_daily.loc[length] = individual_row_data

top_200_track_daily['Extract_Date'] = pd.to_datetime(top_200_track_daily['Extract_Date'])

top_200_update_result = 'Top 200 Update - SUCCESSFUL'
print(top_200_update_result)

# **3. Artist Images and Followers**

In [6]:
def search_for_artists(artist_ids, _th):
    url = f"https://api.spotify.com/v1/artists?ids={artist_ids}"
    token = get_token()
    headers = get_auth_header(token)

    response = get(url, headers=headers)
    
    if response.status_code == 200:
        try:
            json_result = response.json()  # use .json() method directly
            artists = pd.json_normalize(json_result['artists'])
            
            # artist_image table
            artist_info = artists[['id', 'name', 'images']]
            artist_info.rename(columns = {'id': 'Artist_ID', 'name': 'Artist_Name', 'images': 'Artist_Image'}, inplace=True)
            artist_info['Artist_Image'] = artist_info['Artist_Image'].apply(lambda x: x[0]['url'] if x else None)

            # artist_follower table
            artist_follower = artists[['id', 'followers.total']]
            artist_follower.rename(columns = {'id': 'Artist_ID', 'followers.total': 'Followers'}, inplace=True)

            now = datetime.now()
            artist_follower['Extract_Date'] = now.strftime("%Y-%m-%d %H:%M:%S")

            # artist_genres table
            artist_genres = artists[['id', 'genres']]
            artist_genres.rename(columns={'id': 'Artist_ID', 'genres': 'Genre'}, inplace=True)
            
            return artist_info, artist_follower, artist_genres
        
        except Exception as e:
            print(f'Other reason: {e}')
            # Returning None or empty DataFrame to maintain consistency in return type
            return None, None, None

    elif response.status_code == 429:
        print('Too many requests')
        # Returning None or empty DataFrame to maintain consistency in return type
        return None, None, None

    else:
        print(f'Failure - HTTP status code: {response.status_code}')
        # Returning None or empty DataFrame to maintain consistency in return type
        return None, None, None


In [7]:
# Search for existing artist info
artist_info = pd.read_parquet(adls_path + 'artist_info.parquet')
existed_artists = artist_info['Artist_ID'].tolist()

concat_artist_ids = ''
count = 1

artist_image_list = []
artist_follower_list = []
artist_genres_list = []
batch = 50

for artist_id in existed_artists:
    if count % batch in range(1, batch):
        concat_artist_ids = concat_artist_ids + ',' + str(artist_id)
        if count == len(existed_artists):
            batch_th = math.ceil(count / batch)
            concat_artist_ids = concat_artist_ids[1:]
            result = search_for_artists(concat_artist_ids, batch_th)
            
            if result is None:
                artist_search_result = f'Artist Search - FAILED: Batch {batch_th} - Other reasons or too many requests'
                print(artist_search_result)
                break

            updated_artist_image, updated_artist_follower, updated_genres = result
            artist_image_list.append(updated_artist_image)
            artist_follower_list.append(updated_artist_follower)
            artist_genres_list.append(updated_genres)
            
            artist_search_result = f'Artist Search - SUCCESSFUL: {batch_th} batches'
            print(artist_search_result)
    else:
        batch_th = math.floor(count / batch)
        concat_artist_ids = concat_artist_ids + ',' + str(artist_id)
        concat_artist_ids = concat_artist_ids[1:]
        
        result = search_for_artists(concat_artist_ids, batch_th)
        
        if result is None:
            artist_search_result = f'Artist Search - FAILED: Batch {batch_th} - Other reasons or too many requests'
            print(artist_search_result)
            break
        
        updated_artist_image, updated_artist_follower, updated_genres = result
        artist_image_list.append(updated_artist_image)
        artist_follower_list.append(updated_artist_follower)
        artist_genres_list.append(updated_genres)

        artist_search_result = f'Artist Search - SUCCESSFUL: {batch_th} batches'
        print(artist_search_result)

        concat_artist_ids = ''

    count += 1


In [23]:
# Update artist images

try:
    artist_image_update = pd.concat(artist_image_list, ignore_index=True)

    # Find artist with updated images
    compare_image = pd.merge(artist_info, artist_image_update, on='Artist_ID', how='outer')

    # Updated image dataframe if there is update
    change_image_artist = compare_image[compare_image['Artist_Image_x'] != compare_image['Artist_Image_y']]

    # Update image in artist_info df
    if len(change_image_artist) > 0:

        # Loop through all updated artists
        for updated_artist_id in change_image_artist['Artist_ID'].tolist():
            # Get index of updated the row and the updated image
            updated_index = change_image_artist[change_image_artist['Artist_ID'] == updated_artist_id].index.tolist()[0]
            updated_image = change_image_artist.loc[updated_index, 'Artist_Image_y']

            # UPDATE THE IMAGES
            artist_index = artist_info[artist_info['Artist_ID'] == updated_artist_id].index.tolist()[0]
            artist_info.loc[artist_index, 'Artist_Image'] = updated_image

        images_no = len(change_image_artist)
        update_artist_image_result = f'Image Update - SUCCESSFUL: {images_no} images updated'
        print(update_artist_image_result)

    else:
        update_artist_image_result = f'Image Update - SUCCESSFUL: No image updated'
        print(update_artist_image_result)

except:
    update_artist_image_result = f'Image Update - FAILED: Too many requests'
    print(update_artist_image_result)

In [27]:
# Update Artist Follower
artist_follower = pd.read_parquet(adls_path + 'artist_follower.parquet')

try:
    artist_follower_update = pd.concat(artist_follower_list, ignore_index=True)

    # Change columns' names
    artist_follower_update.dropna(inplace=True)
    artist_follower_update['Followers'] = artist_follower_update['Followers'].astype(int)
    artist_follower_update = pd.merge(artist_info, artist_follower_update, on='Artist_ID', how='inner')[['Artist_ID', 'Followers', 'Extract_Date']]
        
    artist_follower = pd.concat([artist_follower, artist_follower_update], axis=0)
    update_artist_follower_result = f'Follower Update - SUCCESSFUL'
    print(update_artist_follower_result)

except:
    update_artist_follower_result = f'Follower Update - FAILED: Too many requests'

In [31]:
artist_follower.info()

# **4. Genres**

In [ ]:
# Add new genre
genres = pd.read_parquet(adls_path + 'genres.parquet')
existed_genres = genres['Genre_Name'].tolist()

try:
    updated_artist_genres = pd.concat(artist_genres_list, ignore_index=True)
    updated_artist_genres = updated_artist_genres.explode('Genre')
    updated_artist_genres.dropna(inplace=True)
    genres_list = list(set(updated_artist_genres['Genre']))

    new_genres = []
    for new_genre in genres_list:
        if new_genre not in existed_genres:
            new_genres.append(new_genre)

    if len(new_genres) > 0:
        for genre_name in new_genres:
            new_genres_id = len(genres) + 1
            new_genre_row = [new_genres_id, genre_name]
            genres.loc[length] = new_genre_row

        no_new_genre = len(new_genres)
        add_genre_result = f'Genres Addition - SUCCESSFUL: {no_new_genre} genres added'
        print(add_genre_result)

    else:
        add_genre_result = f'Genres Addition - SUCCESSFUL: No new genre'
        print(add_genre_result)

except:
    add_genre_result = f'Genres Addition - FAILED: Too many requests'
    print(add_genre_result)

In [ ]:
# Lookup genre code and artist
artist_genres = pd.read_parquet(adls_path + 'artist_genres.parquet')

merge_df = pd.merge(updated_artist_genres, genres, left_on='Genre', right_on='Genre_Name')
artist_genres = merge_df[['Artist_ID', 'Genre_ID']]

# **5. New Artists**

In [ ]:
# If there is new artist in top 200 track, add them into artist info and artist_genres
new_artists = []

for artist_id in list(set(top_200_track_daily['Lead_Artist_ID'])):
    if artist_id not in existed_artists:
        new_artists.append(artist_id)

len(new_artists)

In [ ]:
concat_artist_ids = ''
count = 1

batch = 50

if len(new_artists) > 0:
    for artist_id in new_artists:
        if count % batch in range(1,batch):
            concat_artist_ids = concat_artist_ids + ',' + str(artist_id)
            if count == len(new_artists):
                batch_th = math.ceil(count/batch)
                concat_artist_ids = concat_artist_ids[1:]
                if search_for_artists(concat_artist_ids, batch_th) == 'Failure - Too many requests':
                    artist_search_result = f'Artist Search - FAILED: Batch {batch_th} - Too many requests'
                    print(artist_search_result)
                    break

                elif search_for_artists(concat_artist_ids, batch_th) == 'Failure - Other reasons':
                    artist_search_result = f'Artist Search - FAILED: Batch {batch_th} - Other reasons'
                    print(artist_search_result)
                    break

                else:
                    new_artist_info, new_artist_follower, artist_genres = search_for_artists(concat_artist_ids, batch_th)
                    artist_info = pd.concat([artist_info, new_artist_info], axis=0)
                    artist_follower = pd.concat([artist_follower, new_artist_follower], axis=0)

                    artist_search_result = f'Artist Addition - SUCCESSFUL: {len(new_artists)} Artists added'
                    print(artist_search_result)
                    
        else:
            batch_th = math.floor(count/batch)
            concat_artist_ids = concat_artist_ids + ',' + str(artist_id)
            concat_artist_ids = concat_artist_ids[1:]
            
            if search_for_artists(concat_artist_ids, batch_th) == 'Failure - Too many requests':
                artist_search_result = f'Artist Search - FAILED: Batch {batch_th} - Too many requests'
                print(artist_search_result)
                break

            elif search_for_artists(concat_artist_ids, batch_th) == 'Failure - Other reasons':
                artist_search_result = f'Artist Search - FAILED: Batch {batch_th} - Other reasons'
                print(artist_search_result)
                break
            
            else:
                new_artist_info, new_artist_follower, artist_genres = search_for_artists(concat_artist_ids, batch_th)
                artist_info = pd.concat([artist_info, new_artist_info], axis=0)
                artist_follower = pd.concat([artist_follower, new_artist_follower], axis=0)

                artist_search_result = f'Artist Addition - SUCCESSFUL: {len(new_artists)} Artists added'
                print(artist_search_result)

            concat_artist_ids = ''

        count += 1

else:
    artist_search_result = f'Artist Addition - SUCCESSFUL: No New Artist'
    print(artist_search_result)

In [ ]:
artist_follower['Extract_Date'] = pd.to_datetime(artist_follower['Extract_Date'], format='%Y-%m-%d %H:%M:%S')

# **6. Get New Song Stream**

In [ ]:
col_title = ['Song_ID', 'Stream', 'Extract_Date']
song_stream_updated = pd.DataFrame(columns = col_title)

for artist_id in artist_info['Artist_ID']:
    pattern_url = f'https://kworb.net/spotify/artist/{artist_id}_songs.html'

    page = requests.get(pattern_url)
    soup = BeautifulSoup(page.text, 'html')

    col_data = soup.find_all('tr')

    for row in col_data[5:]:
        row_data = row.find_all('td')
        song_url = row_data[0].find_all('a')[0].get('href')
        pattern = r'https://open.spotify.com/track/([^_]+)'
        match = re.search(pattern, song_url)

        song_id = match.group(1)
        streams = [data.text.strip() for data in row_data][1]
        
        now = datetime.now()
        extract_date = now.strftime("%Y-%m-%d %H:%M:%S")

        individual_row_data = [song_id, streams, extract_date]

        length = len(song_stream_updated)
        song_stream_updated.loc[length] = individual_row_data

# Delete comma in stream number and convert into integer
song_stream_updated['Stream'] = song_stream_updated['Stream'].apply(lambda x: x.replace(",",""))
song_stream_updated['Stream'] = song_stream_updated['Stream'].astype(int)

# Convert Extract date into datetime type
song_stream_updated['Extract_Date'] = pd.to_datetime(song_stream_updated['Extract_Date'], format='%Y-%m-%d %H:%M:%S')

# Delete duplicates
song_stream_updated.drop_duplicates(subset='Song_ID', keep='first', inplace=True)

# **7. New Song Info and Stream**

In [ ]:
def search_for_tracks(track_ids, _th):
    url = f"https://api.spotify.com/v1/tracks?ids={track_ids}"
    token = get_token()
    headers = get_auth_header(token)
    response = get(url, headers=headers)
    
    if response.status_code == 200:
        
        try:
            json_result = response.json()  # use .json() method directly
            songs = pd.json_normalize(json_result['tracks'])
            
            #song_info table
            song_info = songs[['id', 'name', 'explicit', 'duration_ms', 'track_number', 'disc_number']]
            song_info.rename(columns={'id': 'Song_ID',
                                        'name': 'Song_Name',
                                        'explicit': 'Explicit',
                                        'duration_ms': 'Duration',
                                        'track_number': 'Track_Number',
                                        'disc_number': 'Disc_Number'},
                                        inplace=True)

            #artist_song table
            artist_song = songs[['id', 'artists', 'album.artists']]
            artist_song.rename(columns={'id': 'Song_ID', 'artists': 'Artist_ID', 'album.artists': 'Artist_ID_Album'}, inplace=True)
            
            #album_song table
            album_song = songs[['id', 'album.id']]
            album_song.rename(columns={'id': 'Song_ID', 'album.id': 'Album_ID'}, inplace=True)

            return song_info, artist_song, album_song
        
        except:
            other_reason = 'Failure - Other reasons'
            print('Other reasons')
            return other_reason
    
    else:
        failure_retry_after = 'Failure - Too many requests'
        print('Too many requests')
        return failure_retry_after

In [ ]:
song_info = pd.read_parquet(adls_path + 'song_info.parquet')
song_stream = pd.read_parquet(adls_path + 'song_stream.parquet')

merge_df_1 = pd.merge(song_stream_updated, song_stream, on='Song_ID', how='left')
new_tracks_1 = merge_df_1[merge_df_1.isnull().any(axis=1)]['Song_ID'].tolist()

merge_df_2 = pd.merge(top_200_track_daily, song_stream, on='Song_ID', how='left')
new_tracks_2 = merge_df_2[merge_df_2.isnull().any(axis=1)]['Song_ID'].tolist()

new_tracks = new_tracks_1 + new_tracks_2

if len(new_tracks) > 0:
    concat_song_ids = ''
    count = 1

    song_info_list = []
    artist_song_list = []
    album_song_list = []
    batch = 50

    for song_id in new_tracks:
        if count % batch in range(1,batch):
            concat_song_ids = concat_song_ids + ',' + str(song_id)
            if count == len(new_tracks):
                batch_th = math.ceil(count/batch)
                concat_song_ids = concat_song_ids[1:]
                if search_for_tracks(concat_song_ids, batch_th) == 'Failure - Too many requests':
                    track_search_result = f'Track Search - FAILED: Batch {batch_th} - Too many requests'
                    print(track_search_result)
                    break

                elif search_for_tracks(concat_song_ids, batch_th) == 'Failure - Other reasons':
                    track_search_result = f'Track Search - FAILED: Batch {batch_th} - Other reasons'
                    print(track_search_result)
                    break

                else:
                    added_song_info, artist_song, album_song = search_for_tracks(concat_song_ids, batch_th)
                    song_info = pd.concat([song_info, added_song_info], axis=0)

                    artist_song_list.append(artist_song)
                    album_song_list.append(album_song)
                    
                    track_search_result = f'Track Search - SUCCESSFUL: {batch_th} batches'
                    track_addition_result = f'Track Addition - SUCCESSFUL: {len(new_tracks)} tracks added'
                    print(track_search_result)
                    print(track_addition_result)
        else:
            batch_th = math.floor(count/batch)
            concat_song_ids = concat_song_ids + ',' + str(song_id)
            concat_song_ids = concat_song_ids[1:]
            
            if search_for_tracks(concat_song_ids, batch_th) == 'Failure - Too many requests':
                track_search_result = f'Track Search - FAILED: Batch {batch_th} - Too many requests'
                print(track_search_result)
                break
            
            elif search_for_tracks(concat_song_ids, batch_th) == 'Failure - Other reasons':
                track_search_result = f'Track Search - FAILED: Batch {batch_th} - Other reasons'
                print(track_search_result)
                break

            else:
                added_song_info, artist_song, album_song = search_for_tracks(concat_song_ids, batch_th)
                song_info = pd.concat([song_info, added_song_info], axis=0)
                
                artist_song_list.append(artist_song)
                album_song_list.append(album_song)
                
                track_search_result = f'Track Search - SUCCESSFUL: {batch_th} batches'
                track_addition_result = f'Track Addition - SUCCESSFUL: {len(new_tracks)} tracks added'
                print(track_search_result)
                print(track_addition_result)

            concat_song_ids = ''

        count += 1
else:
    track_search_result = f'Track Search - SUCCESSFUL: No track updated'
    print(track_search_result)

In [ ]:
song_info.drop_duplicates(subset='Song_ID', inplace=True)
song_stream = pd.concat([song_stream, song_stream_updated], axis=0)
song_stream_update_result = f'Stream Update - SUCCESSFUL'

# **8. Lead and Feature Artist**

In [ ]:
try:
    # Lead Artist
    added_artist_song = pd.concat(artist_song_list, ignore_index=True)

    def extract_ids(artists):
        return [artist['id'] for artist in artists]

    added_artist_song['Artist_ID'] = added_artist_song['Artist_ID'].apply(extract_ids)
    added_artist_song['Artist_ID_Album'] = added_artist_song['Artist_ID_Album'].apply(extract_ids)

    def find_lead_artists(row):
        return [artist for artist in row['Artist_ID'] if artist in row['Artist_ID_Album']]

    added_artist_song['Lead_Artist_ID'] = added_artist_song.apply(find_lead_artists, axis=1)

    def find_feature_artists(row):
        return [artist for artist in row['Artist_ID'] if artist not in row['Artist_ID_Album']]

    added_artist_song['Feature_Artist_ID'] = added_artist_song.apply(find_feature_artists, axis=1)

    added_lead_artist = added_artist_song[['Lead_Artist_ID', 'Song_ID']]
    added_lead_artist = added_lead_artist.explode('Lead_Artist_ID')
    added_lead_artist.drop_duplicates(inplace=True)

    merge_df = pd.merge(added_lead_artist, artist_info, left_on='Lead_Artist_ID',
                        right_on='Artist_ID', how='left')
    no_lead_artist_rows = merge_df[merge_df['Lead_Artist_ID'].isnull()]['Song_ID'].tolist()
    added_artist_song['Lead_Artist_ID'] = added_artist_song.apply(lambda x: x['Artist_ID'] if x['Song_ID'] in no_lead_artist_rows
                                                else x['Lead_Artist_ID'], axis=1)
    added_artist_song['Feature_Artist_ID'] = added_artist_song.apply(lambda x: [] if x['Song_ID'] in no_lead_artist_rows
                                                    else x['Feature_Artist_ID'], axis=1)
    added_lead_artist = added_artist_song[['Lead_Artist_ID', 'Song_ID']]
    added_lead_artist = added_lead_artist.explode('Lead_Artist_ID')
    added_lead_artist.drop_duplicates(inplace=True)

    merge_df = pd.merge(added_lead_artist, artist_info, left_on='Lead_Artist_ID',
                        right_on='Artist_ID', how='left')
    merge_df.dropna(inplace=True)

    added_lead_artist = merge_df[['Lead_Artist_ID', 'Song_ID']]

    

    # Feature Artist
    added_feature_artist = added_artist_song[['Feature_Artist_ID', 'Song_ID']]
    added_feature_artist = added_feature_artist.explode('Feature_Artist_ID')
    added_feature_artist.drop_duplicates(inplace=True)

    merge_df = pd.merge(added_feature_artist, artist_info, left_on='Feature_Artist_ID',
                        right_on='Artist_ID', how='left')
    merge_df.dropna(inplace=True)

    added_feature_artist = merge_df[['Feature_Artist_ID', 'Song_ID']]
    added_feature_artist = added_feature_artist.reset_index(drop=True)
    

    

except:
    lead_feature_artist_addition_result = 'Lead-Feature Artist Addition - SUCCESSFULL: No artist added'
    print(lead_feature_artist_addition_result)

In [ ]:
# Add into the lead and feature artist
lead_artist = pd.read_parquet(adls_path + 'lead_artist.parquet')
feature_artist = pd.read_parquet(adls_path + 'feature_artist.parquet')

try:
    lead_artist = pd.concat([lead_artist, added_lead_artist], axis=0)
    feature_artist = pd.concat([feature_artist, added_feature_artist], axis=0)

    lead_feature_artist_addition_result = 'Lead-Feature Artist Addition - SUCCESSFULL'
    print(lead_feature_artist_addition_result)

except:
    lead_feature_artist_addition_result = 'Lead-Feature Artist Addition - SUCCESSFULL: No artist added'
    print(lead_feature_artist_addition_result)

# **9. New Album**

In [ ]:
def search_for_albums(album_ids, max_retries=5, backoff_factor=1.0):
    url = f"https://api.spotify.com/v1/albums?ids={album_ids}"
    token = get_token()
    headers = get_auth_header(token)

    for attempt in range(max_retries):
        response = get(url, headers=headers)
        
        if response.status_code == 200:
            
            json_result = response.json()  # use .json() method directly
            albums = pd.json_normalize(json_result['albums'])
            
            #album_info table
            album_info = albums[['id', 'name', 'total_tracks', 'album_type',
                                 'release_date', 'images', 'label']]
            try:
                album_info['images'] = album_info['images'].apply(lambda x: x[0]['url'])
            except:
                album_info['images'] = np.NaN
            album_info.rename(columns={'id': 'Album_ID',
                                        'name': 'Album_Name',
                                        'total_tracks': 'Total_Tracks',
                                        'album_type': 'Album_Type',
                                        'release_date': 'Release_Date',
                                        'images': 'Album_Image',
                                        'label': 'Label'},
                                        inplace=True)

            #artist_album table
            artist_album = albums[['id', 'artists']]
            artist_album['artists'] = artist_album['artists'].apply(extract_ids)
            artist_album = artist_album.explode('artists')
            artist_album.drop_duplicates(inplace=True)
            artist_album.rename(columns={'id': 'Album_ID',
                                        'artists': 'Artist_ID'}, inplace=True)

            return album_info, artist_album
        
        elif response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", backoff_factor * (2 ** attempt)))
            print(f"Too many requests. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
        
        else:
            response.raise_for_status()
    
    raise Exception("Maximum retries exceeded")

In [ ]:
try:
    added_adlbum_song = pd.concat(album_song_list, ignore_index=True)
    new_album = list(set(added_adlbum_song['Album_ID']))

    concat_album_ids = ''
    count = 1

    album_info_list = []
    artist_album_list = []
    batch = 20

    for album_id in new_album:
        if count % batch in range(1,batch):
            concat_album_ids = concat_album_ids + ',' + str(album_id)
            if count == len(new_album):
                batch_th = math.ceil(count/batch)
                concat_album_ids = concat_album_ids[1:]
                if search_for_albums(concat_album_ids, batch_th) == 'Failure - Too many requests':
                    album_search_result = f'Album Search - FAILED: Batch {batch_th} - Too many requests'
                    print(album_search_result)
                    break

                elif search_for_albums(concat_album_ids, batch_th) == 'Failure - Other reasons':
                    album_search_result = f'Album Search - FAILED: Batch {batch_th} - Other reasons'
                    print(album_search_result)
                    break

                else:
                    new_album_info, new_artist_album = search_for_albums(concat_album_ids, batch_th)
                    album_info_list.append(new_album_info)
                    artist_album_list.append(new_artist_album)
                    
                    album_search_result = f'Album Search - SUCCESSFUL: {batch_th} batches'
                    print(album_search_result)

        else:
            batch_th = math.floor(count/batch)
            concat_album_ids = concat_album_ids + ',' + str(album_id)
            concat_album_ids = concat_album_ids[1:]
            
            if search_for_albums(concat_album_ids, batch_th) == 'Failure - Too many requests':
                album_search_result = f'Album Search - FAILED: Batch {batch_th} - Too many requests'
                print(album_search_result)
                break

            elif search_for_albums(concat_album_ids, batch_th) == 'Failure - Other reasons':
                album_search_result = f'Album Search - FAILED: Batch {batch_th} - Other reasons'
                print(album_search_result)
                break

            else:
                new_album_info, new_artist_album = search_for_albums(concat_album_ids, batch_th)
                album_info_list.append(new_album_info)
                artist_album_list.append(new_artist_album)
                
                album_search_result = f'Album Search - SUCCESSFUL: {batch_th} batches'
                print(album_search_result)

            concat_album_ids = ''

        count += 1
        
except:
    album_search_result = f'Album Search - SUCCESSFUL: No album added'
    print(album_search_result)

In [ ]:
# Add new album into album info and artist album
album_info = pd.read_parquet(adls_path + 'album_info.parquet')
artist_album = pd.read_parquet(adls_path + 'artist_album.parquet')
album_song = pd.read_parquet(adls_path + 'album_song.parquet')

try:
    no_new_album = len(new_album)
    new_album_info = pd.concat(album_info_list, ignore_index=True)
    new_artist_album = pd.concat(artist_album_list, ignore_index=True)
    new_album_song = pd.concat(album_song_list, ignore_index=True)

    album_info = pd.concat([album_info, new_album_info], axis=0)
    artist_album = pd.concat([artist_album, new_artist_album], axis=0)
    album_song = pd.concat([album_song, new_album_song], axis=0)

    album_addition_result = f'Album Addition - SUCCESSFUL: {no_new_album} albums added'
    print(album_addition_result)

except:
    album_addition_result = f'Album Addition - SUCCESSFUL: No new album'
    print(album_addition_result)

album_info['Release_Date'] = pd.to_datetime(album_info['Release_Date'])
album_info.drop_duplicates(subset='Album_ID', inplace=True)
album_info = album_info.reset_index(drop=True)
artist_album = artist_album.reset_index(drop=True)
album_song = album_song.reset_index(drop=True)

# **10. Remove Same Song but Different IDs in song_stream**

In [ ]:
song_stream['Date'] = song_stream['Extract_Date'].dt.date
merge_df = pd.merge(song_stream, song_info, on='Song_ID', how='left')[['Song_ID', 'Song_Name', 'Stream', 'Extract_Date', 'Date']]
merge_df.drop_duplicates(subset=['Song_Name', 'Stream', 'Date'], inplace=True)
merge_df = merge_df.reset_index(drop=True)
song_stream = merge_df[['Song_ID', 'Stream', 'Extract_Date']]

# **Exporting Dataset**

In [ ]:
output_file_system_name = 'testing'
storage_account_name = 'spotifyprojectadls'
adls_path = f"abfss://{output_file_system_name}@{storage_account_name}.dfs.core.windows.net/"

In [ ]:
top_200_track_daily.to_parquet(adls_path + 'top_200_track_daily.parquet', index=False)
song_stream.to_parquet(adls_path + 'song_stream.parquet', index=False)
artist_follower.to_parquet(adls_path + 'artist_follower.parquet', index=False)
artist_info.to_parquet(adls_path + 'artist_info.parquet', index=False)
album_info.to_parquet(adls_path + 'album_info.parquet', index=False)
artist_genres.to_parquet(adls_path + 'artist_genres.parquet', index=False)
genres.to_parquet(adls_path + 'genres.parquet', index=False)
lead_artist.to_parquet(adls_path + 'lead_artist.parquet', index=False)
feature_artist.to_parquet(adls_path + 'feature_artist.parquet', index=False)
album_song.to_parquet(adls_path + 'album_song.parquet', index=False)
artist_album.to_parquet(adls_path + 'artist_album.parquet', index=False)
song_info.to_parquet(adls_path + 'song_info.parquet', index=False)